# Summary

<p>In this notebook an audio file in English, part of a poem by the poet Robert Frost, is converted to text using the IBM Speech to Text API. Then the English text is translated to Arabic and Germna  using a Language Translator API. It is then translated backwards to English again.</p>


<p>
The project is inspired by the course "Python Project for AI & Application Development" within the "IBM Applied AI Professional Certificate".<br>
<a href="https://www.coursera.org/professional-certificates/applied-artifical-intelligence-ibm-watson-ai" target="_blank">IBM Coursera</a> 
</p> 



In [1]:
%%capture
#you will need the following library 
!pip install ibm-watson
# !pip install PyJWT==2.0.1 ibm_watson wget

<h2 id="ref0">Speech to Text</h2>


<p>First we import <code>SpeechToTextV1</code> from <code>ibm_watson</code>.For more information on the API, please click on this <a href="https://cloud.ibm.com/apidocs/speech-to-text?code=python">link</a></p>


In [2]:
from ibm_watson import SpeechToTextV1 
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [3]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


In [4]:
import config

## Authentication with service credentials

<p>
"You authenticate to the API by using IBM Cloud Identity and Access Management (IAM)".
</p>

<p>
"You can pass either a bearer token in an authorization header or an API key. Tokens support authenticated requests without embedding service credentials in every call. API keys use basic authentication. For more information, see Authenticating to Watson services."
</p>

<p>
<a href="https://cloud.ibm.com/apidocs/language-translator?code=python" target="_blank">IBM</a> 
</p>

<p>The URL stores the adress of the service endpointL. The service credentials were created on <a href="https://cloud.ibm.com/login" target="_blank">IBM Cloud</a>.</p>


In [5]:
url_s2t = config.url_s2t

In [6]:
iam_apikey_s2t = config.apikey_s2t

<h3>Authentication</h3>
<p>
"You authenticate by providing the API key for your service instance. A Speech to Text: Customer Care cluster has a single instance of the service and a single API key.
The SDK manages the lifecycle of the tokens for the API key."<br>
<a href="https://cloud.ibm.com/docs/watson?topic=watson-iam" target="_blank">IBM Cloud</a> 
</p> 

<p>Further info on parameters are on Github<a href="http://watson-developer-cloud.github.io/python-sdk/v0.25.0/apis/watson_developer_cloud.speech_to_text_v1.html">Speech To Text parameters</a> </p>


In [7]:
authenticator = IAMAuthenticator(iam_apikey_s2t)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

In [8]:
print(authenticator)

<p>The audiofile is already downloaded in the directory.</p>

<p>We have the path of the wav file we would like to convert to text</p>


In [9]:
filename = 'ROBERT_FROST.mp3'

<h3>Audio formats (content types)</h3>
<p>
"The service accepts audio in the following formats (MIME types)."
</p>

<ul>
<li>
For formats that are labeled Required, you must use the   content_type parameter with the request to specify the format of the audio.
</li>
<li>
For all other formats, you can omit the content_type parameter or specify application/octet-stream with the parameter to have the service automatically detect the format of the audio.
</li>
</ul> 

<a href="https://cloud.ibm.com/docs/watson?topic=watson-iam" target="_blank">IBM</a> 










In [10]:
with open(filename, mode="rb")  as wav:
    response = s2t.recognize(audio=wav, content_type='audio/mp3')

The result method of the response class return a dict.<br>
<b>How does the dict look like?</b>

In [11]:
# As a dict
response_result_dict = response.result
response_result_dict

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'here is a brief but apocalyptic poem by Robert Frost which was recently voted the most popular poem among high school students read here by the writer N. Scott Momaday ',
     'confidence': 0.95}]},
  {'final': True,
   'alternatives': [{'transcript': 'fire and ice ', 'confidence': 0.99}]},
  {'final': True,
   'alternatives': [{'transcript': "some say the world will end in fire some say in ice from what I've tasted of desire I hold with those who favor fire but if I had to perish twice I think I know enough of hate to say that for destruction ice is also great and would suffice ",
     'confidence': 0.99}]}]}

In [12]:
response_result_dict.keys()

dict_keys(['result_index', 'results'])

In [13]:
response_result_dict.values()

dict_values([0, [{'final': True, 'alternatives': [{'transcript': 'here is a brief but apocalyptic poem by Robert Frost which was recently voted the most popular poem among high school students read here by the writer N. Scott Momaday ', 'confidence': 0.95}]}, {'final': True, 'alternatives': [{'transcript': 'fire and ice ', 'confidence': 0.99}]}, {'final': True, 'alternatives': [{'transcript': "some say the world will end in fire some say in ice from what I've tasted of desire I hold with those who favor fire but if I had to perish twice I think I know enough of hate to say that for destruction ice is also great and would suffice ", 'confidence': 0.99}]}]])

In [14]:
response.result['results']

[{'final': True,
  'alternatives': [{'transcript': 'here is a brief but apocalyptic poem by Robert Frost which was recently voted the most popular poem among high school students read here by the writer N. Scott Momaday ',
    'confidence': 0.95}]},
 {'final': True,
  'alternatives': [{'transcript': 'fire and ice ', 'confidence': 0.99}]},
 {'final': True,
  'alternatives': [{'transcript': "some say the world will end in fire some say in ice from what I've tasted of desire I hold with those who favor fire but if I had to perish twice I think I know enough of hate to say that for destruction ice is also great and would suffice ",
    'confidence': 0.99}]}]

In [15]:
response.result['results'][0:2]

[{'final': True,
  'alternatives': [{'transcript': 'here is a brief but apocalyptic poem by Robert Frost which was recently voted the most popular poem among high school students read here by the writer N. Scott Momaday ',
    'confidence': 0.95}]},
 {'final': True,
  'alternatives': [{'transcript': 'fire and ice ', 'confidence': 0.99}]}]

In [16]:
response.result['results'][0:2][1]

{'final': True,
 'alternatives': [{'transcript': 'fire and ice ', 'confidence': 0.99}]}

In [17]:
# from pandas import json_normalize
# Why does pd.json_normalize not work?
from pandas.io.json import json_normalize
transcript_df = \
json_normalize(response.result['results'],"alternatives")

print(transcript_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
transcript    3 non-null object
confidence    3 non-null float64
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes
None


In [18]:
transcript_df

,transcript,confidence
0,here is a brief but apocalyptic poem by Robert...,0.95
1,fire and ice,0.99
2,some say the world will end in fire some say i...,0.99


In [19]:
t1 = transcript_df.transcript
t1

0    here is a brief but apocalyptic poem by Robert...
1                                        fire and ice 
2    some say the world will end in fire some say i...
Name: transcript, dtype: object

In [20]:
recognized_text=\
response.result['results'][0]["alternatives"][0]["transcript"]
type(recognized_text)
print(recognized_text)

here is a brief but apocalyptic poem by Robert Frost which was recently voted the most popular poem among high school students read here by the writer N. Scott Momaday 


<h2 id="ref1">Language Translator</h2>


<p>
"IBM Watson™ Language Translator translates text from one language to another. The service offers multiple IBM-provided translation models that you can customize based on your unique terminology and language. Use Language Translator to take news from across the globe and present it in your language, communicate with your customers in their own language, and more."<br>
<a href="https://cloud.ibm.com/apidocs/language-translator?code=python" target="_blank">language-translator</a> 
</p> 

<h3>Package and service credentials.</h3>

In [21]:
from ibm_watson import LanguageTranslatorV3

In [22]:
url_lt = config.url_lt

In [23]:
apikey_lt = config.apikey_lt

<p><b>Versioning</b><br>
"API requests require a version parameter that takes a date in the format version=YYYY-MM-DD. When we change the API in a backwards-incompatible way, we release a new version date."
</p>


In [24]:
version_lt='2018-05-01'

<p>Language Translator object</p>


In [25]:
authenticator = IAMAuthenticator(apikey_lt)
language_translator = LanguageTranslatorV3(version=version_lt,authenticator=authenticator)
language_translator.set_service_url(url_lt)
language_translator

### Preparation
<p>
<b>What languages are offered by the language translator?</b>
</p>


In [26]:
# from pandas import json_normalize
from pandas.io.json import json_normalize
json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")

,language,name
0,af,Afrikaans
1,ar,Arabic
2,az,Azerbaijani
3,ba,Bashkir
4,be,Belarusian
...,...,...
71,uk,Ukrainian
72,ur,Urdu
73,vi,Vietnamese
74,zh,Simplified Chinese


In [27]:
lang_df = json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")
lang_df.head()

,language,name
0,af,Afrikaans
1,ar,Arabic
2,az,Azerbaijani
3,ba,Bashkir
4,be,Belarusian


The abbreviations are needed for the model below.

In [28]:
lang_df[lang_df.name=='Arabic']

,language,name
1,ar,Arabic


In [29]:
lang_df[lang_df.name=='German']

,language,name
12,de,German


### Translation from English to Arabic

<p>The language translator offers a method "translate".
The method needs a model, here: en-ar.</p>


In [30]:
recognized_text

'here is a brief but apocalyptic poem by Robert Frost which was recently voted the most popular poem among high school students read here by the writer N. Scott Momaday '

In [31]:
# recognized_text must be a json
translation_response_arabic = \
language_translator.translate(text=recognized_text, 
                              model_id='en-ar')

In [32]:
translation_arabic = translation_response_arabic.get_result()
translation_arabic

{'translations': [{'translation': 'وهنا قصيدة قصيرة ولكنها غير مقتضبة من قبل روبرت فروست الذي صوت له حديثا القصيدة الأكثر شعبية بين طلاب المدارس الثانوية الذين قرأهم الكاتب "ن. سكوت مالاداي". '}],
 'word_count': 30,
 'character_count': 168}

<p>The keys of the dict are defined in the language translator
class and consistent.</p>


In [33]:
arabic_translation = translation_arabic['translations'][0]['translation']
arabic_translation 

'وهنا قصيدة قصيرة ولكنها غير مقتضبة من قبل روبرت فروست الذي صوت له حديثا القصيدة الأكثر شعبية بين طلاب المدارس الثانوية الذين قرأهم الكاتب "ن. سكوت مالاداي". '

<p>We can translate back to English</p>


In [34]:
backtranslation_engl = \
language_translator.translate(text=arabic_translation, model_id='ar-en').get_result()

backtranslation_engl

{'translations': [{'translation': 'Here is a short poem but not succinct by Robert Frost who recently voted the most popular poem among the high school students read by the writer " N. Scott Maladdai. '}],
 'word_count': 41,
 'character_count': 157}

<p>Only the actual translation text.</p>


In [35]:
backtranslation_engl_text = backtranslation_engl['translations'][0]['translation']
backtranslation_engl_text

'Here is a short poem but not succinct by Robert Frost who recently voted the most popular poem among the high school students read by the writer " N. Scott Maladdai. '

Unfortunately the back translation is not perfect. Some NLP work must still be done on the AI site.

<br>


### Translation from English to German

In [36]:
lang_df[lang_df.name=='German']

,language,name
12,de,German


In [37]:
# recognized_text must be a json
translation_response_german = \
language_translator.translate(text=recognized_text, 
                              model_id='en-de')

In [38]:
translation_german = translation_response_german.get_result()
translation_german

{'translations': [{'translation': 'Hier ist ein kurzes, aber apokalyptisches Gedicht von Robert Frost, das vor kurzem zum beliebtesten Gedicht unter den Gymnasiasten gewählt wurde, das hier vom Schriftsteller N. Scott Momaday gelesen wurde. '}],
 'word_count': 30,
 'character_count': 168}

In [39]:
This translation is good. Will the back translation be good too?

Object `too` not found.


This translation is good. Will the back translation be good too

In [40]:
german_translation = translation_german['translations'][0]['translation']
german_translation

'Hier ist ein kurzes, aber apokalyptisches Gedicht von Robert Frost, das vor kurzem zum beliebtesten Gedicht unter den Gymnasiasten gewählt wurde, das hier vom Schriftsteller N. Scott Momaday gelesen wurde. '

<p>Translation back to English</p>


In [41]:
backtranslation_engl_german = \
language_translator.translate(text=german_translation, model_id='de-en').get_result()

backtranslation_engl_german 

{'translations': [{'translation': 'Here is a short but apocalyptic poem by Robert Frost, who was recently voted the most popular poem among high school students, which was read here by writer N. Scott Momaday. '}],
 'word_count': 34,
 'character_count': 206}

<p>Only the actual translation text.</p>


In [42]:
backtranslation_engl_german_text = backtranslation_engl_german['translations'][0]['translation']
backtranslation_engl_german_text

'Here is a short but apocalyptic poem by Robert Frost, who was recently voted the most popular poem among high school students, which was read here by writer N. Scott Momaday. '

The translation is not perfect, but better than the back translation from Arabic. The similiarity between English and German could make it easier for the AI service. Still some NLP work must still be done on the AI site.

<br>
